In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
!pip install kaggle

In [ ]:
PATH = "data/plant-seedlings-classification"

In [ ]:
!mkdir -p {PATH}

In [ ]:
!export KAGGLE_USERNAME=?? && export KAGGLE_KEY=?? && kaggle competitions download plant-seedlings-classification -p {PATH}

In [ ]:
def create_training_dataset():
    !rm -r {PATH}/test
    !rm -r {PATH}/train
    !rm -r {PATH}/valid
    !unzip {PATH}/test.zip -d {PATH}
    !unzip {PATH}/train.zip -d {PATH}

In [ ]:
create_training_dataset()

In [ ]:
tfms = get_transforms()
data = ImageDataBunch.from_folder(PATH, ds_tfms=tfms, size=224, valid_pct=0.2)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load('stage-1');

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.load('stage-2')

In [ ]:
!pip install onnx_coreml

In [ ]:
import torch
import torch.nn as nn
import numpy as npaa
from onnx_coreml import convert
from torch.autograd import Variable
import torch.onnx
import torchvision
import onnx

In [ ]:
dummy_input = Variable(torch.randn(3, 299, 299)).cuda()

In [ ]:
model_name = "plant_seedlings_classification.onnx"

In [ ]:
torch.onnx.export(learn.model, dummy_input, model_name, \
                  input_names=['image'], output_names=['plantseedlings'], verbose=True)